# Sentinel 2 Spatial Data
Download spatial data and calculate vegetation index

Source:
- [Planetary Computer](https://planetarycomputer.microsoft.com/docs/overview/about)



In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import ipyleaflet

import planetary_computer as pc

import pystac_client
import geopandas as gpd
from satstac import Item, ItemCollection
import intake_stac

import rasterio
import rioxarray
import xarray

import dask
from dask import compute, delayed
from dask.distributed import Client

import datetime
import calendar
import re
import os

pd.set_option("display.max_columns", 999)

In [11]:
client = Client(n_workers=58)

In [4]:
# PARAMETER DEFINITION
BASE_DIR = "./data/monthly ndvi"
AOI = "Schwarzwald"

## Planetary Computer Interface

In [5]:
# get catalog from planetary computer
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")

# explre available collections on planetary computer
collections = catalog.get_children()
for collection in collections:
    if 'landsat' in collection.title.lower() or 'sentinel' in collection.title.lower():
        print(f"{collection.id:<25} \t- {collection.title}")
        
COLLECTION = "sentinel-2-l2a"

collection_childs = catalog.get_child(COLLECTION)
bands = pd.json_normalize(collection_childs.extra_fields["summaries"]["eo:bands"])
bands

sentinel-1-grd            	- Sentinel 1 Level-1 Ground Range Detected (GRD)
sentinel-1-rtc            	- Sentinel 1 Radiometrically Terrain Corrected (RTC)
landsat-8-c2-l2           	- Landsat 8 Collection 2 Level-2
sentinel-2-l2a            	- Sentinel-2 Level-2A
landsat-c2-l1             	- Landsat Collection 2 Level-1
landsat-c2-l2             	- Landsat Collection 2 Level-2


,name,description,gsd,common_name,center_wavelength,full_width_half_max
0,AOT,aerosol optical thickness,NaN,NaN,NaN,NaN
1,B01,coastal aerosol,60.0,coastal,0.443,0.027
2,B02,visible blue,10.0,blue,0.490,0.098
3,B03,visible green,10.0,green,0.560,0.045
4,B04,visible red,10.0,red,0.665,0.038
5,B05,vegetation classification red edge,20.0,rededge,0.704,0.019
6,B06,vegetation classification red edge,20.0,rededge,0.740,0.018
7,B07,vegetation classification red edge,20.0,rededge,0.783,0.028
8,B08,near infrared,10.0,nir,0.842,0.145
9,B8A,vegetation classification red edge,20.0,rededge,0.865,0.033


### Area of Interest
Download data that intersects with AOI

In [6]:
geodf = gpd.read_file(f"./data/ThuenenGeoLocations/geolocations_aoi.geojson")

geodf_aoi = geodf.loc[geodf["bez_wg_bu"] == AOI].iloc[0]

def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)

    return [(min(x_coordinates), min(y_coordinates)), (max(x_coordinates), max(y_coordinates))]

# bbounding box to get data for
points = [point for point in geodf_aoi["geometry"].exterior.coords]
bbox = bounding_box(points)
bbox

[(7.644971139915396, 47.544726784957334),
 (8.836878965890259, 48.9728217552738)]

In [7]:
from pyproj import Proj, transform

def transform_epsg_32632_from_4326(xy):
    """EPSG 32632 from 4326"""
    inProj = Proj(init='epsg:4326')
    outProj = Proj(init='epsg:32632')
    x, y = transform(inProj, outProj, xy[0], xy[1])
    return x, y

def transform_epsg_32632_to_4326(xy):
    """EPSG 32632 from 4326"""
    inProj = Proj(init='epsg:32632')
    outProj = Proj(init='epsg:4326')
    x, y = transform(inProj, outProj, xy[0], xy[1])
    return x, y

BBOX_32632 = list()
BBOX_32632.append(transform_epsg_32632_from_4326(bbox[0]))
BBOX_32632.append(transform_epsg_32632_from_4326(bbox[1]))

# bbox to search data for
BBOX_SEARCH = (bbox[0][0], bbox[0][1], bbox[1][0], bbox[1][1]) # from Blackforest shape import

### BBOX Visual

In [ ]:
m = ipyleaflet.Map(scroll_wheel_zoom=False, center=(geodf_aoi["geometry"].centroid.xy[1][0], geodf_aoi["geometry"].centroid.xy[0][0]), zoom=7.25, height="400px")

# add controls
m.add_control(ipyleaflet.ScaleControl(position="bottomleft"))
m.add_control(ipyleaflet.FullScreenControl())

# define polygon
polygon_coords = np.array(geodf_aoi["geometry"].exterior.coords.xy)
polygon_locations = list()
for i in range(0, len(polygon_coords[1])):
    polygon_locations.append((polygon_coords[1][i], polygon_coords[0][i]))

polygon = ipyleaflet.Polygon(
    locations=polygon_locations,
    color="green",
    fill_color="green"
)
m.add_layer(polygon)

# define bbox around polygon
rectangle = ipyleaflet.Rectangle(bounds=((bbox[0][1], bbox[0][0]), (bbox[1][1], bbox[1][0])), color="blue", weight=3, fill_opacity=0)
m.add_layer(rectangle)

m

#### Identify Tiles

In [9]:
time_range = "2016-05-01/2016-08-31"

search = catalog.search(
    collections=[COLLECTION], 
    bbox=BBOX_SEARCH, 
    datetime=time_range
)

# save search results into geojson file
search.get_all_items().save_object(f"{BASE_DIR}/GeoJSON/{AOI}_all_tiles.geojson")
# load geojson file to get all attributes
gf = gpd.read_file(f"{BASE_DIR}/GeoJSON/{AOI}_all_tiles.geojson")

tiles = gf.groupby("s2:mgrs_tile").agg({"geometry": lambda x:x.value_counts().index[0]}).reset_index()
for _i, tile in tiles.iterrows():
    polygon_coords = np.array(tile.geometry.exterior.coords.xy)
    polygon_locations = list()
    for i in range(0, len(polygon_coords[1])):
        polygon_locations.append((polygon_coords[1][i], polygon_coords[0][i]))

    # build rectangle
    polygon_tile = ipyleaflet.Polygon(
        locations=polygon_locations,
        color="red",
        weight=2,
        fill_opacity=0
    )
    # plot rectangle as new layer on map
    m.add_layer(polygon_tile)

# tiles that needs to be fetched as they interact with bbox of AOI
TILES_OF_INTEREST = list(tiles.loc[tiles.intersects(geodf_aoi["geometry"])]["s2:mgrs_tile"].unique())
print(f"Tiles of interest for selected area: {' '.join(TILES_OF_INTEREST)}")

Tiles of interest for selected area: 32TLT 32TMT 32ULU 32UMU 32UMV


## Data Preparation
- Download data
- Remove clouds
- Calculate vegetation index
- Save data by month
- Combine data by year
- Clip AOI
- Save AOI

parallize using DASK

In [10]:
def get_scenes(month: str, year: str, bbox: tuple)->pystac_client.item_search.ItemSearch:
    last_month_day = calendar.monthrange(int(year), int(month))[-1]
    time_range = f"{year}-{month}-01/{year}-{month}-{last_month_day}"

    search = catalog.search(
        collections=[COLLECTION], 
        bbox=bbox, 
        datetime=time_range
    )

    search.get_all_items().save_object(f"{BASE_DIR}/GeoJSON/{AOI}_{month}_{year}.geojson")

    print("-"*75)
    print(f'Total: {len([i for i in search.get_items()])} matches for {month}-{year}')
    print("-"*75)
    return search

@dask.delayed
def calc_vegetation_index(intake_stac_scene: intake_stac.catalog.StacItem)->(xarray.Dataset, str):
    try:
        if os.path.exists(f"{BASE_DIR}/scenes/{intake_stac_scene.name}.nc"):
            ds = xarray.open_dataset(f"{BASE_DIR}/scenes/{intake_stac_scene.name}.nc", decode_coords="all").rio.write_crs(32632)
        else:
            # Create DataArray using Raster data from RED and NIR band
            da_red = rioxarray.open_rasterio(pc.sign(intake_stac_scene.B04.metadata["href"]))
            da_nir = rioxarray.open_rasterio(pc.sign(intake_stac_scene.B08.metadata["href"]))
            scl = rioxarray.open_rasterio(pc.sign(intake_stac_scene.SCL.metadata["href"]))

            scl_high = scl.reindex(x=da_nir.x, y=da_nir.y, method='nearest')
            no_cloud_map = (scl_high[0] != 8) & (scl_high[0] != 9) & (scl_high[0] != 10) & (scl_high[0] != 3)

            # Calculate NDVI from float32 (f4) arrays
            nir_values = da_nir.values[0].astype('f4')
            red_values = da_red.values[0].astype('f4')
            ndvi = (nir_values - red_values) / (nir_values + red_values)
            evi2 = 2.5 * (nir_values - red_values) / (nir_values + 2.4 * red_values + 1.0)
            # include nan after these have been lost due to evi2 calculation
            evi2 = np.where(~np.isnan(ndvi), evi2, np.nan)
            
            ds = xarray.Dataset(
                data_vars=dict(
                    ndvi=(["x", "y"], np.where(no_cloud_map, ndvi, np.nan)),
                    evi2=(["x", "y"], np.where(no_cloud_map, evi2, np.nan))
                ),
                coords=dict(
                    x=(["x"], da_nir.x.values),
                    y=(["y"], da_nir.y.values)
                )
            )
            
            #ds = ds.rio.write_crs(int(str(da_nir.rio.crs).split(":")[-1]))
            ds = ds.rio.write_crs(32632)
            
            #if intake_stac_scene.metadata["date"].month == 5 and intake_stac_scene.metadata["date"].year == 2016:
            #    ds.to_netcdf(f"{BASE_DIR}/scenes/{intake_stac_scene.name}.nc", 'w', engine='netcdf4')

            # return dataset with ndvi and evi2 and scene date
            del ndvi, evi2, da_red, da_nir, nir_values, red_values, scl, scl_high, no_cloud_map
        
        # rio clip bounding box
        ds = ds.rio.clip_box(
            minx=BBOX_32632[0][0],
            miny=BBOX_32632[0][1],
            maxx=BBOX_32632[1][0],
            maxy=BBOX_32632[1][1],
        )
        
        return ds, intake_stac_scene.metadata["date"]
    except (rasterio.RasterioIOError):
        return None, None
    
def process_scenes(intakeStacItemCollection: intake_stac.catalog.StacItemCollection)->xarray.Dataset:
    #scenes = []
    date_scenes = []
    ds_scenes = []

    delayed_tasks = []
    for s in range(len(list(intakeStacItemCollection))):
        print(f'Current scene ({s}) {list(intakeStacItemCollection)[s]} started at {str(datetime.datetime.now())}')
        delayed_tasks.append(calc_vegetation_index(intakeStacItemCollection[list(intakeStacItemCollection)[s]]))
    
    processed_index = compute(*delayed_tasks)
    
    ds_scenes, date_scenes = zip(*processed_index)

    ds_scenes = list(filter(None, ds_scenes))
    date_scenes = list(filter(None, date_scenes))

    for i, ds in enumerate(ds_scenes):
        if ds["ndvi"].shape[0] == 0 or ds["ndvi"].shape[1] == 0:
            del ds_scenes[i]
            del date_scenes[i]

    #ds_concat_max = xarray.concat([ds_scene.to_array(name="vegetation index", dim="index") for ds_scene in ds_scenes], dim=xarray.Variable('date', pd.to_datetime(date_scenes))).to_dataset(dim="index").groupby("date.month").max()
    ds_concat_max = xarray.concat(ds_scenes, dim=xarray.Variable('date', pd.to_datetime(date_scenes))).groupby('date.month').max()

    # RAM mgmt
    del ds_scenes, date_scenes

    return ds_concat_max

def merge_year_scenes_by_tile(year: int):
    """merge scene file parts by tile to year vegetation index (still separated by tile)"""
    month_files = {}
    dates = {}
    tiles = {}
    
    for file in os.listdir(f"{BASE_DIR}/month"):
        # regex match using AOI from variable
        m = re.match("".join([AOI, "\_(\d{2})\_(\d{4})\_(\d{2}\w{3})\.nc"]), file)
        
        if m:
            month = m.group(1)
            y = m.group(2)
            tile = m.group(3)
            if y not in month_files:
                month_files[y] = {}
                dates[y] = {}
                tiles[y] = {}

                month_files[y][tile] = [f"{BASE_DIR}/month/{file}"]
                dates[y][tile] = [f"01-{month}-{y}"]
                tiles[y][tile] = [tile]
            else:
                if tile not in month_files[y]:
                    month_files[y][tile] = [f"{BASE_DIR}/month/{file}"]
                    dates[y][tile] = [f"01-{month}-{y}"]
                    tiles[y][tile] = [tile]
                else:
                    month_files[y][tile].append(f"{BASE_DIR}/month/{file}")
                    dates[y][tile].append(f"01-{month}-{y}")
                    tiles[y][tile].append(tile)
    
    tile_datasets = []
    for tile in month_files[year]:
        tile_datasets.append(
            xarray.concat(
                [xarray.open_dataset(file, decode_coords="all").squeeze("month").drop("month") for file in month_files[year][tile]], # squeeze month dimension before
                dim=xarray.Variable('date', pd.to_datetime(dates[year][tile])) # concatenating datasets by date (month)
            ).groupby("date.year").max().expand_dims({"tile": tiles[year][tile]}).groupby("tile").max()
        )

    ds = xarray.concat(tile_datasets, dim="tile")

    del tile_datasets

    ds.to_netcdf(f"{BASE_DIR}/year/{AOI}_{year}.nc", 'w', engine='netcdf4')

    del ds, month_files, dates, tiles


In [ ]:
# months and years to loop through
months = [f"{month:02d}" for month in range(5, 9)] # may to august
years = [str(year) for year in range(2015, 2022)] # 2015 to 2021

for year in years:
    for month in months:
        # get scenes from planetary computer
        search = get_scenes(month=month, year=year, bbox=BBOX_SEARCH)
        # create pyStacItemCollection
        pyStacItemCollection = search.get_all_items()
        
        items_list = np.array([item.to_dict() for item in pyStacItemCollection])
        
        if len(items_list) == 0:
            print(f"No scenes found for {month}-{year} ...")
            continue
        else:
            for _i, tile in enumerate(TILES_OF_INTEREST):
                item_list_tile_part = list()
                # identify scenes from same tile
                for item_in_list in items_list:
                    if tile in item_in_list["id"]:
                        item_list_tile_part.append(item_in_list)

                print("-"*50)
                print(f"Starting iteration for tile {tile} ({_i})")
                print("-"*50)

                # pystac to stacstac item
                # necessary to use within intake_stac collection
                stacStacItems = [Item(sentinel_item) for sentinel_item in item_list_tile_part]

                # items to stac item collection
                stacStacItemCollection = ItemCollection(stacStacItems)

                # StacItemCollection to filter Catalog by Item Name
                intakeStacItemCollection = intake_stac.catalog.StacItemCollection(stacStacItemCollection)

                # get dataset (max index values)
                ds = process_scenes(intakeStacItemCollection)
                # export to nc file
                ds.to_netcdf(f"{BASE_DIR}/month/{AOI}_{month}_{year}_{tile}.nc", 'w', engine='netcdf4')

                del ds

    print("Calculating max vegetation index for current year...")
    merge_year_scenes_by_tile(str(year))
    
    print(f"Done ({year}) ...")
    print()
